# **Projeto Quality Checker Kyndryl**
 
## **Made by Vìtor Ribeiro**

In [1]:
import re
import json
from datetime import datetime

# Instalação de libs caso não estejam na máquina do desenvolvedor
try:
    import pandas as pd
except ImportError:
    %pip install pandas
finally:
    import pandas as pd

try:
    from pyfiglet import Figlet
except ImportError:
    %pip install pyfiglet
finally:
    from pyfiglet import Figlet

try:
    from distutils.util import strtobool
except ImportError:
    %pip install python3-distutils
finally:
    from distutils.util import strtobool

In [2]:
f = Figlet(width=150)
print(f.renderText("Script made by\nVitor R. G. Gomes - Kyndryl\n2021-2022"))

 ____            _       _                         _        _           
/ ___|  ___ _ __(_)_ __ | |_   _ __ ___   __ _  __| | ___  | |__  _   _ 
\___ \ / __| '__| | '_ \| __| | '_ ` _ \ / _` |/ _` |/ _ \ | '_ \| | | |
 ___) | (__| |  | | |_) | |_  | | | | | | (_| | (_| |  __/ | |_) | |_| |
|____/ \___|_|  |_| .__/ \__| |_| |_| |_|\__,_|\__,_|\___| |_.__/ \__, |
                  |_|                                             |___/ 
__     ___ _               ____       ____      ____                                     _  __               _            _ 
\ \   / (_) |_ ___  _ __  |  _ \     / ___|    / ___| ___  _ __ ___   ___  ___          | |/ /   _ _ __   __| |_ __ _   _| |
 \ \ / /| | __/ _ \| '__| | |_) |   | |  _    | |  _ / _ \| '_ ` _ \ / _ \/ __|  _____  | ' / | | | '_ \ / _` | '__| | | | |
  \ V / | | || (_) | |    |  _ < _  | |_| |_  | |_| | (_) | | | | | |  __/\__ \ |_____| | . \ |_| | | | | (_| | |  | |_| | |
   \_/  |_|\__\___/|_|    |_| \_(_)  \____(_)  \____|\___/|_| 

In [ ]:
df = pd.read_excel("./incident_merged.xlsx")

pd.set_option("display.max_colwidth", None)
df.head()
df7 = df.groupby("Number")
len(df7)

In [ ]:
df2 = df[(df['Short description'].str.contains('autosys job|ZEN',case=False)==False) & (df['Work notes'].str.contains('autosys job|ZEN',case=False)==False)]
df3 = df2.groupby("Number")
len(df3)

In [ ]:
def checkCallList(entry):
    entry = entry.lower()
    if "call list:" and "job name:" and "instructions:" in entry:
        return True
    return False

In [ ]:
def checkTimeLine(entry):
    entry = entry.lower()
    timeLineDateArray = []
    workNotesTechArray = entry.split("\\n\\n")
    appendArray = False
    for pos in workNotesTechArray:
        if "email sent to" in pos and '@' in pos:
            match = re.search(
                "^([0-9]{4}-[0-9]{2}-[0-9]{2}\s[0-9]{2}:[0-9]{2}:[0-9]{2}(\.[0-9]{1,3})?)",
                pos,
            )
            if match:
                appendArray = True

        elif "call" in pos and re.search(
            "^(([0-9]{3}) |[0-9]{3}-)[0-9]{3}-[0-9]{4}$", pos
        ):
            match = re.search(
                "^([0-9]{4}-[0-9]{2}-[0-9]{2}\s[0-9]{2}:[0-9]{2}:[0-9]{2}(\.[0-9]{1,3})?)",
                pos,
            )
            if match:
                appendArray = True
        elif ("sent pager" or "pager sent") in pos and '@' in pos or re.search("^(([0-9]{3}) |[0-9]{3}-)[0-9]{3}-[0-9]{4}$", pos):
            match = re.search(
                "^([0-9]{4}-[0-9]{2}-[0-9]{2}\s[0-9]{2}:[0-9]{2}:[0-9]{2}(\.[0-9]{1,3})?)",
                pos,
            )
            if match:
                appendArray = True
        if appendArray:
            timeLineDateArray.append(
                datetime.strptime(match.group(0)[2:], "%y-%m-%d %H:%M:%S")
            )
        appendArray = False

    return timeLineDateArray

In [ ]:
def checkCloseNotes(entry):
    entry = entry.lower()
    
    rcaTitle = ["RCA", "Root Cause", "rootcause"]
    rcaTitle = [element.lower() for element in rcaTitle]
    rcaT = False
    for value in rcaTitle:
        if value in entry:
            rcaT = True
            break

    rcaDetail = [
        "Job Fail",
        "Job Failed",
        "Job Failure",
        "Job got faIled",
        "Autosys alarm",
        "Failed",
        "max run",
        "maxrun",
        "maxrunalarm",
        "generated due",
        "max run alarm",
        "max time",
        "maxtime",
        "abend",
        "ran longer",
        "killed",
        "unsuccessful",
        "job alert",
        "automated ritm",
        "ABENDED",
        "Job automation failure",
    ]
    rcaDetail = [element.lower() for element in rcaDetail]
    rcaD = False
    for value in rcaDetail:
        if value in entry:
            rcaD = True
            break

    actionTitle = [
        "Action taken",
        "steps taken",
        "actiontaken",
        "stepstaken",
        "resolution",
        "actions taken",
    ]
    actionTitle = [element.lower() for element in actionTitle]
    actionT = False
    for value in actionTitle:
        if value in entry:
            actionT = True
            break

    actionTakenActions = [
        "Restarted",
        "force started",
        "force success",
        "force completed",
        "duplicated alert",
        "mark as success",
        "mark job success",
        "ON ICE",
        "ON HOLD",
        "JOB_ON_HOLD",
        "CANCELLED",
        "complete",
        "completed",
        "success status",
        "success",
        "kill",
        "killed",
    ]
    actionTakenActions = [element.lower() for element in actionTakenActions]
    actionTakenA = False
    for value in actionTakenActions:
        if value in entry:
            actionTakenA = True
            break

    actionTakenRequests = [
        "as per",
        "by",
        "with the comment",
        "in a new cycle",
        "RITM",
        "as requested by",
        "completed successfully",
        "per email",
        "INC",
        "asked to",
        "reset",
    ]
    actionTakenRequests = [element.lower() for element in actionTakenRequests]
    actionTakenR = False
    for value in actionTakenRequests:
        if value in entry:
            actionTakenR = True
            break

    nextStepsTitle = [
        "Next Steps",
        "Future Steps",
        "Permanent Fix",
        "Next Step",
        "Further Action",
    ]
    nextStepsTitle = [element.lower() for element in nextStepsTitle]
    nextT = False
    for value in nextStepsTitle:
        if value in entry:
            nextT = True
            break

    nextStepsDetail = ["None", "NA", "N/A"]
    nextStepsDetail = [element.lower() for element in nextStepsDetail]
    nextS = False
    for value in nextStepsDetail:
        if value in entry:
            nextS = True
            break

    if (rcaT and rcaD and actionT and actionTakenA and actionTakenR and nextT and nextS):
        if "completed successfully" in entry and not ("max run" in entry or "maxrun" in entry
        or "maxrunalarm" in entry or "max run alarm" in entry):
            return False
        elif "INC" in entry and not (re.search("INC[0-9]{8,12}", entry) and "duplicated alert" in entry):
            return False
        elif "RITM" in entry and not (re.search("RITM[0-9]{7}", entry)):
            return False
        return True
    return False

In [ ]:
def checkEscalation(entry, group):
    if not entry:
        return False
    else:
        for i in range(group.shape[0]):
            df4 = group.iloc[[i]]
            if (
                df4["Value"].to_string(index=False).strip()
                == "Awaiting Technical Information"
            ):
                start = datetime.strptime(
                    df4["Start"].to_string(index=False).strip()[2:], "%y-%m-%d %H:%M:%S"
                )
                for date in entry:
                    if start > date and (start - date).seconds > 1800:
                        return False
                    start = date
                return True

In [ ]:
employeesJson = json.load(open("./employees.json"))
cont=0
for key in df3.groups.keys():  
    df4 = df3.get_group(key).iloc[[df3.get_group(key).shape[0] - 1]]
    assignedTo = df4["Assigned To"].to_string(index=False).strip()
    incidentNumber = df4["Number"].to_string(index=False).strip()
    print(cont,incidentNumber)
    made_sla = bool(strtobool(df4["Made SLA"].to_string(index=False).strip()))

#     callList = checkCallList(df4["Work notes"].to_string(index=False).strip())
    timeLineCheck = checkTimeLine(df4["Work notes"].to_string(index=False).strip())
    closeNotes = checkCloseNotes(df4["Resolution Notes"].to_string(index=False).strip())

    error = False
    errorArray = []

    # Para fazer a checagem de Escalation, é passado todas as timeLines encontradas

    # Entrada para forçar o erro durante a etapa de escalation
    #     timeLineErrada = []
    #     timeLineErrada.append(datetime.strptime("21-09-30 23:00:01", "%y-%m-%d %H:%M:%S"))
    #     escalation = checkEscalation(timeLineErrada,df2.get_group(key))

    # Entrada correta
    escalation = checkEscalation(timeLineCheck, df3.get_group(key))

    for idx, employee in enumerate(employeesJson):
        if assignedTo == employee["operator"]:
            if made_sla:
                employeesJson[idx]["totalSuccesses"] += 1
            else:
                employeesJson[idx]["totalMistakes"] += 1
                employeesJson[idx]["mistakesDetails"][0]["NumberOfMistakes"] += 1
                error = True
                errorArray.append(employeesJson[idx]["mistakesDetails"][0]["name"])
#             if callList:
#                 employeesJson[idx]["totalSuccesses"] += 1
#             else:
#                 employeesJson[idx]["totalMistakes"] += 1
#                 employeesJson[idx]["mistakesDetails"][1]["NumberOfMistakes"] += 1
#                 error = True
#                 errorArray.append(employeesJson[idx]["mistakesDetails"][1]["name"])

            # Se não houver timeLine achada, já existe erro de escalation, pelo menos 1 timeline deve existir sempre
            if timeLineCheck:
                employeesJson[idx]["totalSuccesses"] += 1
                if escalation:
                    employeesJson[idx]["totalSuccesses"] += 1
                else:
                    employeesJson[idx]["totalMistakes"] += 1
                    employeesJson[idx]["mistakesDetails"][2]["NumberOfMistakes"] += 1
                    error = True
                    errorArray.append(employeesJson[idx]["mistakesDetails"][2]["name"])
            else:
                # Itera erro de TimeLine
                employeesJson[idx]["totalMistakes"] += 1
                employeesJson[idx]["mistakesDetails"][3]["NumberOfMistakes"] += 1
                # Itera erro de Escalation por tabela
                employeesJson[idx]["totalMistakes"] += 1
                employeesJson[idx]["mistakesDetails"][2]["NumberOfMistakes"] += 1
                error = True
                errorArray.append(employeesJson[idx]["mistakesDetails"][3]["name"])
                errorArray.append(employeesJson[idx]["mistakesDetails"][2]["name"])
            if closeNotes:
                employeesJson[idx]["totalSuccesses"] += 1
            else:
                employeesJson[idx]["totalMistakes"] += 1
                employeesJson[idx]["mistakesDetails"][4]["NumberOfMistakes"] += 1
                error = True
                errorArray.append(employeesJson[idx]["mistakesDetails"][4]["name"])
            employeesJson[idx]["totalWorkedIncidents"] += 1
            if(error):
                employeesJson[idx]["incidentsWithError"].append({
                    "number": incidentNumber,
                    "typeOfErrors":errorArray
                })

            cont+=1
        

jsonFinal = json.dumps(employeesJson, indent=4)
print(jsonFinal)

In [ ]:
#Porcentagem de Incidents analisado com sucesso
print(cont)
print(len(df3.groups.keys()))
percent = cont*100/len(df3.groups.keys())
print(percent)

In [ ]:
json.dump(employeesJson,open("./results.json","w"),indent=2)

# **Análise Gráfica dos Resultados de cada operador**

In [ ]:
resultsJson = json.load(open("./results.json"))
len(resultsJson)

In [ ]:
import matplotlib.pyplot as plt
resultsJson = json.load(open("./results.json"))
operator = resultsJson[11]['operator']
totalMistakes = resultsJson[11]['totalMistakes']
totalSuccesses = resultsJson[11]['totalSuccesses']

In [ ]:
print(totalMistakes)
print(totalSuccesses)

In [ ]:
f = plt.figure()
f.set_figwidth(10)
f.set_figheight(1)

In [ ]:
plt.pie([totalMistakes,totalSuccesses],autopct='%0.1f%%',textprops={'fontsize': 18}) 
plt.legend(labels=['Mistakes','Successes'],fontsize=18)
plt.title(operator+' Overall',fontsize=18)
plt.show();